# Task Settings

In [1]:
import os
os.environ['OPENAI_API_KEY']=""

from utils import *
from prompts import Prompts, TASK_LABLES, TAGS

dataset_name = 'swiss'                      # 'trafficsafe', 'pamdemic'
test_model = "experimental:gpt-4o-mini"     # Forward Engine
eval_model = "gpt-4o"                       # Backward Engine
iteration = 1
date = '0919'
total_steps=9                               # 5-11
epoch=1                                     # 2, 1
batch_size=3                                # 1-3

# Initialize

In [2]:
cm_labels = TASK_LABLES[dataset_name]
tags = TAGS[dataset_name]
CAUSAL_SYSTEM = Prompts[dataset_name]['CAUSAL_SYSTEM']
CAUSAL_SYSTEM_CONSTRAINT = Prompts[dataset_name]['CAUSAL_SYSTEM_CONSTRAINT']
SYSTEM = Prompts[dataset_name]['SYSTEM']

llm_api_eval = tg.get_engine(engine_name=eval_model)
llm_api_test = tg.get_engine(engine_name=test_model, cache=False)
tg.set_backward_engine(llm_api_eval, override=True)

train_set, val_set, test_set_ori, eval_fn = load_task(dataset_name, evaluation_api=llm_api_eval, prompt_col="organized_prompt")
train_loader = tg.tasks.DataLoader(train_set, batch_size=batch_size, shuffle=True)
col = "organized_prompt" if dataset_name == 'swiss' else "prompt"
train_set.data[col] = train_set.data[col].apply(lambda x: f"{tags[0]}{x}{tags[1]}")
val_set.data[col] = val_set.data[col].apply(lambda x: f"{tags[0]}{x}{tags[1]}")
test_set_ori.data[col] = test_set_ori.data[col].apply(lambda x: f"{tags[0]}{x}{tags[1]}")
print("Train/Val/Test Set Lengths: ", len(train_set), len(val_set), len(test_set_ori))

Train/Val/Test Set Lengths:  168 84 85


In [3]:
import matplotlib.pyplot as plt
from copy import deepcopy


test_set = deepcopy(test_set_ori)
system_prompt, causal_prompt, model, causal_model, optimizer, optimizer_causal = init(SYSTEM, CAUSAL_SYSTEM, llm_api_test, llm_api_eval, CAUSAL_SYSTEM_CONSTRAINT)
results = {"test_f1": [], "prompt": [], "validation_f1": [], 'system_prompt':[], 'causal_prompt': []}

In [4]:
acc, t_rs, t_ys = eval_dataset(test_set, eval_fn, model,iters=1)
test_res = evaluate_metrics(t_rs, t_ys, cm_labels)

print(test_res)

Accuracy: 0.4235: 100%|██████████| 85/85 [00:10<00:00,  8.25it/s]

{'accuracy': 0.4235294117647059, 'precision': 0.580392156862745, 'f1': 0.42376741041244087, 'cm': array([[24,  2,  1],
       [20,  8,  1],
       [10,  2,  4]])}


In [5]:
acc, t_rs, t_ys = eval_dataset(test_set, eval_fn, model,iters=1)
test_res = evaluate_metrics(t_rs, t_ys, cm_labels)

print(test_res)

Accuracy: 0.4471: 100%|██████████| 85/85 [00:09<00:00,  8.82it/s]

{'accuracy': 0.4470588235294118, 'precision': 0.5125730500805277, 'f1': 0.40861040068201193, 'cm': array([[28,  1,  1],
       [20,  8,  2],
       [11,  4,  2]])}


In [6]:
acc, t_rs, t_ys = eval_dataset(test_set, eval_fn, model,iters=1)
test_res = evaluate_metrics(t_rs, t_ys, cm_labels)

print(test_res)

Accuracy: 0.4000: 100%|██████████| 85/85 [02:00<00:00,  1.42s/it]

{'accuracy': 0.4, 'precision': 0.5140756302521008, 'f1': 0.39028386462623077, 'cm': array([[23,  2,  3],
       [20,  8,  1],
       [13,  2,  3]])}


# Run EGO-Prompt

In [4]:
import time
import copy
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed

NUM_WORKERS = 1 

def run_one_worker(worker_id: int):


    local_test_set = copy.deepcopy(test_set)
    local_test_set_ori = copy.deepcopy(test_set_ori)
    local_val_set = copy.deepcopy(val_set)
    local_train_loader = train_loader

    val_performance = -float('inf')
    test_performance = -float('inf')
    final_results = None
    all_val_f1s = []
    all_test_f1s = []

    local_test_set.data[col] = local_test_set_ori.data[col].apply(
        lambda x: f"<!-- {time.time()} (w{worker_id}) -->, {x}"
    )

    for cur_iter in range(iteration):
        print(f"[Worker {worker_id}] [Iteration {cur_iter+1}/{iteration}] begin")

        output_json = (
            f"res/{date}_{dataset_name}_{test_model.split('/')[-1].split(':')[-1]}_"
            f"w{worker_id}_it{cur_iter+1}.json"
        )
        initialize_json_file(output_json)

        system_prompt, causal_prompt, model, causal_model, optimizer, optimizer_causal = init(
            SYSTEM, CAUSAL_SYSTEM, llm_api_test, llm_api_eval, CAUSAL_SYSTEM_CONSTRAINT
        )

        results, test_res, val_res = init_eval(
            local_val_set, local_test_set, eval_fn, model, causal_model,
            system_prompt, causal_prompt, cm_labels, iters=ITERS    
        )

        results = run_training(
            local_train_loader, local_val_set, local_test_set, eval_fn,
            model, causal_model, system_prompt, causal_prompt,
            optimizer, optimizer_causal, results, cm_labels,
            output_json=output_json, epoch=epoch, steps=total_steps, iters=ITERS
        )

        all_val_f1s.append(results['validation_f1'])
        all_test_f1s.append(results['test_f1'])

        cur_val = results['validation_f1'][-1]
        cur_test = results['test_f1'][-1]
        if cur_val > val_performance:
            val_performance = cur_val
            test_performance = cur_test
            final_results = results

        print(f"[Worker {worker_id}] [Iteration {cur_iter+1}] "
              f"val_best={val_performance:.4f}, test_at_best={test_performance:.4f}")

    return {
        'best_test_f1': test_performance,
        'val_f1s': all_val_f1s,
        'test_f1s': all_test_f1s,
        'worker_id': worker_id,
    }

# Running the cell below will incur API usage charges. Refer to our paper for detailed cost

In [5]:
NUM_WORKERS = 3
ITERS = 1
total_steps=5

EGO_res = []


with ThreadPoolExecutor(max_workers=NUM_WORKERS) as ex:
    futures = [ex.submit(run_one_worker, i) for i in range(NUM_WORKERS)]
    for fut in as_completed(futures):
        res = fut.result()
        EGO_res.append(res['best_test_f1'])
        print(f"[Main] Worker {res['worker_id']} done. Best test_f1={res['best_test_f1']:.4f}")

print("EGO_res (best test F1 per worker):", EGO_res)

[Worker 0] [Iteration 1/1] begin[Worker 1] [Iteration 1/1] begin

[Worker 2] [Iteration 1/1] begin


Accuracy: 0.1364:  24%|██▍       | 20/84 [00:07<00:07,  8.33it/s]

SCG_val_f1: 0.3182539682539683, SCG_test_f1:0.20487520070062765


Accuracy: 0.1304:  27%|██▋       | 23/84 [00:07<00:06,  9.95it/s]


Epoch 0, Step 0


...

Skip Test
[Causal Validation] F1: 0.4336, Previous F1: 0.4679
[Causal Validation CM]:
[[15  8 10]
 [16 15  1]
 [ 8  4  6]]
Skip Test
[Worker 1] [Iteration 1] val_best=0.4679, test_at_best=0.4604
[Main] Worker 1 done. Best test_f1=0.4604


Accuracy: 0.5176: 100%|██████████| 85/85 [00:19<00:00,  4.26it/s]
5it [28:16, 339.35s/it]
Accuracy: 0.3478:  26%|██▌       | 22/84 [00:14<00:08,  7.29it/s]

[Test Result] F1: 0.5262
[Worker 2] [Iteration 1] val_best=0.4690, test_at_best=0.5262
[Main] Worker 2 done. Best test_f1=0.5262


Accuracy: 0.5119: 100%|██████████| 84/84 [00:22<00:00,  3.70it/s]
5it [28:29, 341.85s/it]

Skip Test
[Causal Validation] F1: 0.4907, Previous F1: 0.5435
[Causal Validation CM]:
[[18 11  4]
 [11 22  0]
 [ 9  6  3]]
Skip Test
[Worker 0] [Iteration 1] val_best=0.5435, test_at_best=0.4879
[Main] Worker 0 done. Best test_f1=0.4879
EGO_res (best test F1 per worker): [0.46038415366146457, 0.5261764705882354, 0.4878954248366013]
